Entrar no venv `.\tcc\Scripts\activate.bat`. Download Opencv em [Source Forge](https://sourceforge.net/projects/opencvlibrary/files/opencv-win/3.4.1/opencv-3.4.1-vc14_vc15.exe/download) e no caminho `build/x64/vc15/bin` possui os executáveis necessários.

Com os executáveis, agora basta configurar o Dataset. Imagens negativas obtidas (8000 imagens) do Imagenet e imagens positivas que são só o corpo dos resistores. As imagens negativas são renomeadas e listadas em um txt com o seguinte script `renameNegatives.py`:

In [ ]:
import os

# caminho para o diretório de imagens
diretorio = "data_set/negatives"

# lista os arquivos
arquivos = os.listdir(diretorio)

imagens = [arquivo for arquivo in arquivos]

for indice, imagem in enumerate(imagens, start=1):
    extensao = os.path.splitext(imagem)[1]
    novo_nome = f"{indice}{extensao}"
    caminho_antigo = os.path.join(diretorio, imagem)
    caminho_novo = os.path.join(diretorio, novo_nome)
    os.rename(caminho_antigo, caminho_novo)
    print(f"{imagem} renomeado para {novo_nome}")

# atualizar a lista de arquivos após renomeação
arquivos = os.listdir(diretorio)

# salvar a lista em um arquivo .txt
with open("negatives/negatives.txt", "w") as arquivo:
    for nome in arquivos:
        caminho_completo = os.path.join(diretorio, nome)
        if os.path.isfile(caminho_completo):  # verifica se é um arquivo
            arquivo.write(f"{nome}\n")

print(f"Lista de arquivos salva")

: 

Devido a erros de ROI na geração das imagens (no texto escrever garantir mesma resolução), é necessário executar o script `removeImagesDiffRes.py` que remove imagens que não estão no padrão de 224x224. Isso resultou na exclusão de 8000-6217 = 1783 imagens. Com isso, podemos executar o `opencv_createsamples.exe` passando as informações necessárias, resultando na geração de 1800 imagens positivas:

In [ ]:
execs_opencv\opencv_createsamples -img data_set/positives/resistor1.jpg -bg data_set/negatives/negatives.txt -info data_set/positives/positives.lst -maxxangle 0.5 -maxyangle 0.5 -maxzangle 0.5 -num 1800 -bgcolor 255 -bgthresh 8

In [ ]:
execs_opencv\opencv_createsamples -info data_set/positives/positives.lst -num 1800 -w 25 -h 25 -vec data_set/positives/positives.vec

Esse penis desse comando não funciona atualmente.

In [ ]:
execs_opencv\opencv_traincascade -data classificador -vec data_set/positives/positives.vec -bg data_set/negatives/negatives.txt -numPos 1300 -numNeg 800 -numStages 10 -precalcBufSize 4096 -precalcIdxBufSize 4096